<a href="https://colab.research.google.com/github/werowe/HypatiaAcademy/blob/master/class/sqllite3_create_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inventory System

* products
* vendors

# Sales System

* customers
* sales transactions

# SQL Has no API

For example, Google would have an API to connect to Big Query.  That would let you add, update, or delete data.

But most databases also support SQL (Structured Query Language).


**Referential Integrity**--for example, if you have orders for customer number 3 and customer number 3 does not exist then you have a mess.

Not logical consistent.

In [23]:
#The first step is to mount all of this on Google Drive using this cell here.

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
#load the SQL extension

%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [25]:
#connect to database on Google Drive

%sql sqlite:////content/drive/MyDrive/inventory.db

In [26]:
# Enable foreign key constraints

%sql PRAGMA foreign_keys = ON;

 * sqlite:////content/drive/MyDrive/inventory.db
Done.


[]

In [27]:
# I added this to delete the customers and orders table as a first step.
# This is because if we want to change the tables the first thing to do is to delete them if they already exist.

%%sql
drop table orders;


 * sqlite:////content/drive/MyDrive/inventory.db
(sqlite3.OperationalError) no such table: orders
[SQL: drop table orders;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [28]:
%%sql
drop table customers

 * sqlite:////content/drive/MyDrive/inventory.db
Done.


[]

In [29]:
#CREATING THE CUSTOMERS TABLE


#import the SQLite library

import sqlite3


#define the path to the SQLite database file

db_path = '/content/drive/MyDrive/inventory.db'


#conn(short for connect) to the SQLite database
#if the database doesn't exist, it will be created

conn = sqlite3.connect(db_path)


#put on foreign key support in SQLite
#it will not create a customer order without the specific customer number
conn.execute('PRAGMA foreign_keys = ON')


#the cursor allows us to interact with the database by executing SQL statements
cursor = conn.cursor()


#SQL command to create a table, which will store all the info about our customers
create_parent_table = '''
CREATE TABLE customers
  (
    customernumber INTEGER PRIMARY KEY AUTOINCREMENT,
    customername text,
    phonenumber text,
    postalcode text,
    datecreated int,
    email text
  );
'''


try:
    #execute the SQL command to create the "customers" table
    cursor.execute(create_parent_table)


except Exception as e:
    #this will print any errors that we might get
    print(e)


#saving the changes
conn.commit()


#stop the connection with our database
conn.close()



In [30]:
#ADDING NEW CUSTOMERS


#connect to the database once more
conn = sqlite3.connect(db_path)

#put on the foreign key support
conn.execute('PRAGMA foreign_keys = ON')

#the cursor allows us to interact with the database by executing SQL statements
cursor = conn.cursor()

#this command adds a customer with specific details
sql='''
INSERT INTO customers (customername, phonenumber, postalcode, datecreated, email)
VALUES ('John Doe', '555-1234', '12345', strftime('%s', 'now'), 'johndoe@example.com');
'''

#execute the SQL command to insert the new customer
cursor.execute(sql)



#save the changes
conn.commit()

#close the connection
conn.close()



In [31]:
#inserts information about a new customer into our database

%%sql
INSERT INTO customers (customername, phonenumber, postalcode, datecreated, email)
VALUES ('Fred Smith', '555-1234', '12345', strftime('%s', 'now'), 'fred@example.com');


 * sqlite:////content/drive/MyDrive/inventory.db
1 rows affected.


[]

In [32]:
#EXECUTE A QUERY TO GET ALL RECORDS FROM THE CUSTOMERS TABLE

import sqlite3

#connect to the SQLite database
conn = sqlite3.connect(db_path)

#create a cursor
cursor = conn.cursor()

#SQL command to select all rows from the customers table
select_query = 'SELECT * FROM customers'

#execute the query
cursor.execute(select_query)

#this will return all records from the "customers" table
#turns the cursor into a list
rows = cursor.fetchall()

#check if rows were found and print them
if rows:
    print("Customer Records:")
    for row in rows:
        print(row)
else:
    print("No records found.")

#close the connection
conn.close()

Customer Records:
(1, 'John Doe', '555-1234', '12345', 1728055408, 'johndoe@example.com')
(2, 'Fred Smith', '555-1234', '12345', 1728055408, 'fred@example.com')


In [33]:
#run SQL query to select all records from the "customers" table

%%sql
SELECT * FROM customers;

 * sqlite:////content/drive/MyDrive/inventory.db
Done.


customernumber,customername,phonenumber,postalcode,datecreated,email
1,John Doe,555-1234,12345,1728055408,johndoe@example.com
2,Fred Smith,555-1234,12345,1728055408,fred@example.com


In [41]:
#ADDING THE "ORDERS" TABLE


import sqlite3


conn = sqlite3.connect(db_path)

cursor = conn.cursor()

create_table_query = '''
CREATE TABLE orders (
    customernumber TEXT NOT NULL,
    ordernumber INTEGER PRIMARY KEY AUTOINCREMENT,
    comments TEXT,
    orderdate TEXT,
    ordertype TEXT,
    shipdate TEXT,
    discount REAL,
    quantity INTEGER,
    productnumber TEXT
    price INTEGER,
    FOREIGN KEY (customernumber) REFERENCES customers(customernumber)
);
'''

try:
  cursor.execute(create_table_query)

except Exception as e:
  print(e)

conn.commit()

conn.close()


In [42]:

%%sql
insert into orders(customernumber,
    quantity,
    productnumber)
  values(2,123,105);

 * sqlite:////content/drive/MyDrive/inventory.db
1 rows affected.


[]

In [43]:
#customer 4 does not exist, that is why we get an error

%%sql
insert into orders(customernumber,ordernumber,quantity, productnumber)
  values(4,11, 8,123);

 * sqlite:////content/drive/MyDrive/inventory.db
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: insert into orders(customernumber,ordernumber,quantity, productnumber)
  values(4,11, 8,123);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [44]:
%%sql

SELECT * FROM orders;

 * sqlite:////content/drive/MyDrive/inventory.db
Done.


customernumber,ordernumber,comments,orderdate,ordertype,shipdate,discount,quantity,productnumber
2,1,None,None,None,None,None,123,105


In [45]:
#command to view the tables we created

%%sql
PRAGMA table_info(orders);

 * sqlite:////content/drive/MyDrive/inventory.db
Done.


cid,name,type,notnull,dflt_value,pk
0,customernumber,TEXT,1,None,0
1,ordernumber,INTEGER,0,None,1
2,comments,TEXT,0,None,0
3,orderdate,TEXT,0,None,0
4,ordertype,TEXT,0,None,0
5,shipdate,TEXT,0,None,0
6,discount,REAL,0,None,0
7,quantity,INTEGER,0,None,0
8,productnumber,TEXT price INTEGER,0,None,0


In [46]:
#command to view the tables we created

%%sql
PRAGMA table_info(customers);


 * sqlite:////content/drive/MyDrive/inventory.db
Done.


cid,name,type,notnull,dflt_value,pk
0,customernumber,INTEGER,0,None,1
1,customername,TEXT,0,None,0
2,phonenumber,TEXT,0,None,0
3,postalcode,TEXT,0,None,0
4,datecreated,INT,0,None,0
5,email,TEXT,0,None,0
